# Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class



In [1]:
import pandas as pd

df = pd.read_csv('ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
def classify_rating(rating_val):
    rating_val = float(rating_val)
    if rating_val <= 2.0:
        return 'низкий рейтинг'
    elif rating_val <= 4.0:
        return 'средний рейтинг'
    elif rating_val in [4.5, 5.0]:
        return 'высокий рейтинг'
    return 'скорее высокий рейтинг, но по условию точно не сказано'

df['class'] = df['rating'].apply(classify_rating)
df.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


In [5]:
df.sort_values('rating', ascending=False)

,userId,movieId,rating,timestamp,class
33889,242,2929,5.0,956687566,высокий рейтинг
36867,265,1233,5.0,960056214,высокий рейтинг
46251,337,1356,5.0,1447176421,высокий рейтинг
14749,95,3948,5.0,1025556197,высокий рейтинг
14747,95,3917,5.0,1019023102,высокий рейтинг
...,...,...,...,...,...
76709,531,1258,0.5,1243454822,низкий рейтинг
2530,15,109187,0.5,1465794437,низкий рейтинг
76706,531,1183,0.5,1244062924,низкий рейтинг
76700,531,1036,0.5,1240402328,низкий рейтинг


# Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.



In [40]:
import re

class GeoClassifier:
    def __init__(self, rules):
        """
        Parameters:
        rules (dict) - словарь вида <региог>: ['город', ...]
        """
        self.geo_rules = {}
        # ага, это не супер эффективно, но захотелось через класс сделать
        for k in rules:
            # если плохие нахвания городов будут - будет ломаться регулярка, но задача учебная
            self.geo_rules[k] = re.compile( '(\W|^)({})(\W|$)'.format( '|'.join(rules[k]).lower() ) )
        
    def classify(self, string):
        for k in self.geo_rules:
            if self.geo_rules[k].match(string.lower()):
                return k
        return 'undefined'


In [37]:
df = pd.read_csv('keywords.csv')
df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [41]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] 
}

gc = GeoClassifier(geo_data) # not garbage collector =)

df['region'] = df['keyword'].apply(lambda x: gc.classify(x))

In [44]:
df[df.region != 'undefined'].head()

,keyword,shows,region
1259,москва,153050,Центр
5858,владивосток,43038,Дальний Восток
5983,москва третий рим,43182,Центр
6075,ярославль,41720,Центр
6999,москва 24,37459,Центр


# Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [46]:
ratings_df = pd.read_csv('ratings.csv')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [59]:
movies_df = pd.read_csv('movies.csv')
movies_df = movies_df.set_index('movieId')
movies_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [66]:
def production_year(title, years = range(1950,2011)):
    for y in years:
        if str(y) in title:
            return y
    return 1900

movies_df['year'] = movies_df['title'].apply(production_year)
movies_df.head(6)

,title,genres,year
movieId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,Jumanji (1995),Adventure|Children|Fantasy,1995
3,Grumpier Old Men (1995),Comedy|Romance,1995
4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
5,Father of the Bride Part II (1995),Comedy,1995
6,Heat (1995),Action|Crime|Thriller,1995


In [69]:
ratings_df['year'] = ratings_df['movieId'].apply(lambda mid: movies_df.loc[mid].year)
ratings_df.head()

,userId,movieId,rating,timestamp,year
0,1,31,2.5,1260759144,1995
1,1,1029,3.0,1260759179,1900
2,1,1061,3.0,1260759182,1996
3,1,1129,2.0,1260759185,1981
4,1,1172,4.0,1260759205,1989


In [77]:
# средний рейтинг по годам (по убыванию)
ratings_df.groupby('year')['rating'].mean().sort_values(ascending=False)

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
          ...   
2005    3.448434
2003    3.444777
1996    3.426600
1997    3.415764
2000    3.356104
Name: rating, Length: 62, dtype: float64